<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/4_30_Robust_Sharp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#入力パラメータ


#Classical MVP

In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha0_classical_mv_unconstrained_target.py  –  α = 0 （古典的平均–分散ポートフォリオ、和の制約なし、目標リターンあり）
    • sum pi = 1 の制約を外し、目標リターン mu_tilde を達成しつつ分散を最小化するポートフォリオを計算
    • alpha_sweep_K4M4_SR.csv と互換な列構成で 1 行だけの CSV を生成
    • SR_rob には計算されたポートフォリオの Sharpe を格納
    • w*, λ, dpi* は NaN で空欄（robust 問題に固有の列のため）
"""

# ---------------------------------------------------------------------
# 0. IMPORTS & CONSTANTS
# ---------------------------------------------------------------------
import numpy as np
import pandas as pd

K, M        = 4, 4
mu_tilde    = 0.03           # 目標リターン (必要)
alpha0      = 0.0            # 古典設定

# ---------------------------------------------------------------------
# 1. SYNTHETIC DATA（元スクリプトと同一）
# ---------------------------------------------------------------------
r_base = np.array([0.02, 0.03, 0.04, 0.05])
Delta_A = np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -2, +2, -2],
                    [+2, -2, +2, -2]], dtype=float)
Delta_r_m     = Delta_A / 1000
sigma_base    = np.array([0.20, 0.25, 0.30, 0.35])
Delta_sigma_m = Delta_A / 100

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j:
                C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j:
                C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
tilde_C_list = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]

def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4):
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval_clip = np.clip(eigval, eps, None)
    C_spd = eigvec @ np.diag(eigval_clip) @ eigvec.T
    D = np.sqrt(np.diag(C_spd))
    return C_spd / np.outer(D, D)

def build_params(alpha: float):
    """Return mean matrix R (K×M) and second-moment matrices Σ (K×K×M)."""
    R = r_base[:, None] + alpha * Delta_r_m
    Sigma = np.zeros((K, K, M))
    for m in range(M):
        sig_vec = sigma_base + alpha * Delta_sigma_m[:, m]
        diag_sig = np.diag(sig_vec)
        C_alpha = mix_corr(C_base, tilde_C_list[m], alpha, beta=0.5)
        Sigma[:, :, m] = diag_sig @ C_alpha @ diag_sig + np.outer(R[:, m], R[:, m])
    return R, Sigma

# ---------------------------------------------------------------------
# 2. α = 0 の古典 MVP を厳密に解く（和の制約なし、目標リターン制約あり）
# ---------------------------------------------------------------------
R0, Sigma0_arr = build_params(alpha0)

# モデル重みは一様  w_m = 1/M
w_uniform = np.ones(M) / M
# 平均リターンベクトルは、各モデルの期待リターンの平均とする
r_bar     = R0 @ w_uniform

# 共分散行列は、古典設定ではモデル0のデータから計算（元のスクリプトに従う）
Cov_bar   = Sigma0_arr[:,:,0]-np.outer(R0[:,0],R0[:,0])

# ---- 目標リターン mu_tilde を達成し、分散を最小化（和の制約なし）----
# 最適解は pi = lambda * inv(Cov_bar) @ r_bar
# 制約: r_bar^T @ pi = mu_tilde
# r_bar^T @ (lambda * inv(Cov_bar) @ r_bar) = mu_tilde
# lambda * (r_bar^T @ inv(Cov_bar) @ r_bar) = mu_tilde
# lambda = mu_tilde / (r_bar^T @ inv(Cov_bar) @ r_bar)

invC = np.linalg.inv(Cov_bar)

# B_prime = r_bar^T @ invC @ r_bar  (これは元のスクリプトのCに等しい)
B_prime = r_bar @ invC @ r_bar

# lambda を計算 (ただし B_prime がゼロでないことを仮定)
if B_prime < 1e-9: # ゼロに近い場合のエラー回避
     raise ValueError("r_bar @ inv(Cov_bar) @ r_bar is close to zero. Cannot achieve target return.")

lam = mu_tilde / B_prime

# 最適ポートフォリオベクトル (和の制約なし)
pi_star_unconstrained = lam * invC @ r_bar

# このポートフォリオベクトルの期待リターン (mu_tilde )
expected_ret_star = r_bar @ pi_star_unconstrained

# このポートフォリオベクトルの分散
var_star_unconstrained = pi_star_unconstrained @ Cov_bar @ pi_star_unconstrained

# 計算されたポートフォリオの Sharpe Ratio (リスクフリーレート0)
# SR = Expected Return / sqrt(Variance)
# Expected Return は mu_tilde に設定されている
# SR = mu_tilde / sqrt(var_star_unconstrained)
# var_star_unconstrained = (lambda * invC @ r_bar)^T @ Cov_bar @ (lambda * invC @ r_bar)
# = lambda^2 * (invC @ r_bar)^T @ Cov_bar @ (invC @ r_bar)
# = lambda^2 * (r_bar^T @ invC @ Cov_bar @ invC @ r_bar)
# = lambda^2 * (r_bar^T @ invC @ r_bar) = lambda^2 * B_prime
# SR = mu_tilde / sqrt(lambda^2 * B_prime) = mu_tilde / (abs(lambda) * sqrt(B_prime))
# lambda = mu_tilde / B_prime なので
# SR = mu_tilde / (abs(mu_tilde / B_prime) * sqrt(B_prime))
# If mu_tilde > 0, SR = mu_tilde / ((mu_tilde / B_prime) * sqrt(B_prime)) = B_prime / sqrt(B_prime) = sqrt(B_prime)
# If mu_tilde < 0, SR = mu_tilde / ((-mu_tilde / B_prime) * sqrt(B_prime)) = -B_prime / sqrt(B_prime) = -sqrt(B_prime)
# Generally, SR = sign(mu_tilde) * sqrt(B_prime)

SR_calc = np.sign(mu_tilde) * np.sqrt(B_prime) if B_prime > 0 else 0 # B_prime <= 0 の場合はSR定義が問題になる

# ---- λ_min ----
lambda_min_cov = np.min(np.linalg.eigvalsh(Cov_bar))

# ---------------------------------------------------------------------
# 3. CSV 生成 – α-sweep 互換フォーマット
# ---------------------------------------------------------------------
cols_w   = [f"w*_m{m}"   for m in range(M)]
cols_pi  = [f"pi*_k{k}"  for k in range(K)]
cols_ret = [f"cstr_ret_m{m}" for m in range(M)]
cols_lam = [f"lam_m{m}"  for m in range(M)] # ここでの lam は上記の lambda
cols_dpi = [f"dpi*_k{k}" for k in range(K)]

columns = (["alpha", "H_star", "supp_w", "lambda_min",
            "iterations", "SR_rob", "SR_bound"]
           + cols_w + cols_pi + cols_ret + cols_lam + cols_dpi)

# lam_m は robust 問題の各モデルに対するラグランジュ乗数だが、
# ここでは和の制約なしMV問題の単一のラグランジュ乗数 lambda を格納する
# フォーマット互換性のため、lambda を cols_lam の最初の要素に入れる
lam_output = [lam] + [np.nan] * (len(cols_lam) - 1)


row = [alpha0,
       var_star_unconstrained,                    # H_star = 計算されたポートフォリオベクトルの分散
       np.nan,                                  # supp_w   – 固定しないので空欄
       lambda_min_cov,                          # lambda_min of Cov_bar
       np.nan,                                  # iterations – 求解回数無し
       SR_calc,                                 # SR_rob (計算されたポートフォリオのSharpe)
       np.sqrt(B_prime) if B_prime > 0 else np.nan] # SR_bound (理論上限、sqrt(C) または sqrt(B_prime))
row += [np.nan] * len(cols_w)                   # w*  — robust 特有 → NaN
row += list(pi_star_unconstrained)              # pi* 4 つ (和の制約なし、目標リターン達成ベクトル)
# 各モデル m = 0..M-1 における計算されたポートフォリオの期待リターン
row += list(R0.T @ pi_star_unconstrained)       # 各モデル制約リターン
row += lam_output                               # λ   — 単一の lambda を格納
row += [np.nan] * len(cols_dpi)                 # dpi* — NaN

df = pd.DataFrame([row], columns=columns)
df.to_csv("alpha0_classical_mv_unconstrained_target.csv", index=False, float_format="%.10g")
print("CSV written to alpha0_classical_mv_unconstrained_target.csv")

CSV written to alpha0_classical_mv_unconstrained_target.csv


#Robust MBP $r^m ,\sigma^m,C^m$全てが異なる場合

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha_sweep_K4M4_SR.py  –  Robust MVP α-sweep  (K = M = 4)
2025-04-30  Sharpe-export version

・最適解 (π*, w*) に対する
      – ロバスト Sharpe 比   SR_rob
      – 理論上限             SR_bound = √[ρ_max /(1–ρ_max)]
  を CSV に追記する。
元スクリプト（2025-04-28 patched version）の Q1–Q5 修正は維持。
"""

# ---------------------------------------------------------------------
# 0. IMPORTS & GLOBAL CONSTANTS
# ---------------------------------------------------------------------
import numpy as np
import pandas as pd
import itertools, logging, sys
from datetime import datetime

K, M         = 4, 4
mu_tilde     = 0.03
ALPHA_MIN    = 0.01
ALPHA_MAX    = 0.5
N_ALPHA      = 501
ALPHA_GRID   = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)

MAX_OPT_ITERS            = 1000
tol_grad                 = 1e-8
step_size_norm_threshold = 1e-12
lr_init, lr_max          = 0.15, 1.0

FINITE_DIFF_EPS          = 1e-12

# ---------------------------------------------------------------------
# 1. SYNTHETIC DATA
# ---------------------------------------------------------------------
r_base = np.array([0.02, 0.03, 0.04, 0.05])
Delta_A = np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -2, +2, -2],
                    [+2, -2, +2, -2]], dtype=float)
Delta_r_m     = Delta_A / 1000
sigma_base    = np.array([0.20, 0.25, 0.30, 0.35])
Delta_sigma_m = Delta_A / 100

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j:
                C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j:
                C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
tilde_C_list = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]
beta_corr = 1

def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4):
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval_clip = np.clip(eigval, eps, None)
    C_spd = eigvec @ np.diag(eigval_clip) @ eigvec.T
    D = np.sqrt(np.diag(C_spd))
    return C_spd / np.outer(D, D), float(eigval_clip.min())

# ---------------------------------------------------------------------
# 2. SIMPLEX PROJECTION
# ---------------------------------------------------------------------
def proj_simplex(v: np.ndarray) -> np.ndarray:
    v = np.asarray(v, float)
    if (v >= 0).all() and np.isclose(v.sum(), 1.0, atol=1e-9):
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - 1
    rho = np.where(u - cssv / (np.arange(len(u)) + 1) > 0)[0][-1]
    theta = cssv[rho] / (rho + 1)
    return np.maximum(v - theta, 0.0)

# ---------------------------------------------------------------------
# 3. PARAMETER CONSTRUCTION
# ---------------------------------------------------------------------
def build_params(alpha: float):
    R_th = r_base[:, None] + alpha * Delta_r_m
    Sigma_th = np.zeros((K, K, M))
    lambda_log = []
    for m in range(M):
        sig_vec = sigma_base + alpha * Delta_sigma_m[:, m]
        diag_sig = np.diag(sig_vec)
        C_alpha, lam_min = mix_corr(C_base, tilde_C_list[m], alpha,
                                    beta=beta_corr)
        lambda_log.append(lam_min)
        Sigma_th[:, :, m] = diag_sig @ C_alpha @ diag_sig + np.outer(R_th[:, m], R_th[:, m])
    return R_th, Sigma_th, lambda_log

# ---------------------------------------------------------------------
# X.  ROBUST SHARPE と 上限を計算
# ---------------------------------------------------------------------
def robust_sharpe(pi, w, R, Sigma):
    """SR_rob と理論上限 SR_bound を返す。"""
    if pi is None or w is None:
        return np.nan, np.nan
    r_bar   = R @ w
    Sigma_b = sum(w[m] * Sigma[:, :, m] for m in range(M))
    Vw      = Sigma_b - np.outer(r_bar, r_bar)
    num     = pi @ r_bar
    denom   = np.sqrt(pi @ Vw @ pi)
    SR_rob  = num / denom if denom > 0 else np.nan
    # ρ_max
    rho_max = max(
        R[:, m] @ np.linalg.inv(Sigma[:, :, m]) @ R[:, m]
        for m in range(M)
    )
    SR_bound = (np.sqrt(rho_max / (1 - rho_max))
                if rho_max < 1 else np.nan)
    return SR_rob, SR_bound

# ---------------------------------------------------------------------
# 4. INNER MINIMISATION  (exact solve over active subsets)
# ---------------------------------------------------------------------
def inner_opt(w, R, Sigma,
              tol_feas   = 1e-9,
              lam_tol    = -1e-13,
              rank_scale = 1e-10):
    rank_tol = rank_scale * np.linalg.norm(R)
    Ex = R @ w
    Vw = sum(w[m] * Sigma[:, :, m] for m in range(M)) - np.outer(Ex, Ex)
    try:
        Vinv = np.linalg.inv(Vw)
    except np.linalg.LinAlgError:
        return None, np.inf, Ex, None, None

    best_val = np.inf
    best_pi = best_lam = best_subset = None

    for s in range(1, M + 1):
        for subset in itertools.combinations(range(M), s):
            A = R[:, subset].T
            if np.linalg.matrix_rank(A, tol=rank_tol) < s:
                continue
            Mmat = A @ Vinv @ A.T
            if np.linalg.matrix_rank(Mmat, tol=rank_tol) < s:
                continue
            lam = np.linalg.solve(Mmat, mu_tilde * np.ones(s))
            if np.any(lam < lam_tol):
                continue
            pi = Vinv @ A.T @ lam
            if np.any(R.T @ pi < mu_tilde - tol_feas):
                continue
            val = pi @ Vw @ pi
            if val + 1e-12 < best_val:
                best_val, best_pi, best_lam, best_subset = val, pi, lam, subset

    if best_pi is None:
        return None, np.inf, Ex, None, None
    return best_pi, best_val, Ex, best_lam, best_subset

# ---------------------------------------------------------------------
# 5. OUTER OBJECTIVE & GRADIENT
# ---------------------------------------------------------------------
def H_and_grad(w, R, Sigma):
    pi, val, Ex, lam, subset = inner_opt(w, R, Sigma)
    if val == np.inf:
        return val, None, pi, lam, subset
    grad = np.array([
        pi @ Sigma[:, :, m] @ pi - 2 * (pi @ R[:, m]) * (pi @ Ex)
        for m in range(M)
    ])
    return val, grad, pi, lam, subset

# ---------------------------------------------------------------------
# 6. OUTER OPTIMISER (PG + BB + Armijo)
# ---------------------------------------------------------------------
def optimise_outer(R, Sigma, *, start=None, seed=None):
    if start is None:
        if seed is None:
            w = np.ones(M) / M
        else:
            rng = np.random.default_rng(seed)
            w = proj_simplex(rng.random(M))
    else:
        w = proj_simplex(start)

    val, g, pi, lam, subset = H_and_grad(w, R, Sigma)
    best = (val, w.copy(), pi, lam, subset)
    lr   = lr_init
    g_prev, w_prev = g.copy(), w.copy()

    for it in range(1, MAX_OPT_ITERS + 1):
        proj_grad = w - proj_simplex(w - g)
        if np.linalg.norm(proj_grad) < tol_grad:
            break

        if it > 1:
            dw, dg = w - w_prev, g - g_prev
            denom  = np.dot(dg, dg)
            if denom > 1e-12:
                gamma_bb = np.dot(dw, dg) / denom
                if gamma_bb > 0.0:
                    lr = np.clip(gamma_bb, 1e-4, lr_max)

        lr_curr = lr
        accept  = False
        while lr_curr >= 1e-6:
            w_trial = proj_simplex(w + lr_curr * g)
            if np.linalg.norm(w_trial - w) < step_size_norm_threshold:
                lr_curr = 0.0
                break
            val_t, g_t, pi_t, lam_t, subset_t = H_and_grad(w_trial, R, Sigma)
            if val_t >= val - 1e-12:
                w_prev, g_prev = w.copy(), g.copy()
                w, val, g = w_trial, val_t, g_t
                pi, lam, subset = pi_t, lam_t, subset_t
                if val > best[0]:
                    best = (val, w.copy(), pi, lam, subset)
                lr = min(lr_curr * 1.2, lr_max)
                accept = True
                break
            lr_curr *= 0.5
        if not accept:
            break

    return (*best, it)

# ---------------------------------------------------------------------
# 7. FINITE DIFFERENCE HELPERS
# ---------------------------------------------------------------------
def finite_diff_vec(v_p, v_m, h, length):
    if v_p is None or v_m is None:
        return np.full(length, np.nan)
    return (v_p - v_m) / max(h * 2, FINITE_DIFF_EPS)

# ---------------------------------------------------------------------
# 8. α-SWEEP
# ---------------------------------------------------------------------
def robust_opt(alpha, *, seed=None):
    R_th, Sigma_th, _ = build_params(alpha)
    H_star, w_star, pi_star, lam_star, subset_star, iters = optimise_outer(
        R_th, Sigma_th, seed=seed)
    return dict(H_star=H_star, w_star=w_star, pi_star=pi_star,
                lam_star=lam_star, subset=subset_star, R=R_th, Sigma=Sigma_th,
                iters=iters)

def run_alpha_sweep(csv_path="alpha_sweep_K4M4_SR.csv", *, seed_outer=None):
    log = logging.getLogger("sweep")
    log.setLevel(logging.INFO)
    log.addHandler(logging.StreamHandler(sys.stdout))

    col_w   = [f"w*_m{m}"   for m in range(M)]
    col_pi  = [f"pi*_k{k}"  for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]
    col_lam = [f"lam_m{m}"  for m in range(M)]
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha", "H_star", "supp_w", "lambda_min", "iterations",
                "SR_rob", "SR_bound"]               # 追加列
               + col_w + col_pi + col_ret + col_lam + col_dpi)

    rows, cache = [], {}
    t0 = datetime.now()
    log.info("========== α-Sweep START ==========")

    for idx, alpha in enumerate(ALPHA_GRID):
        log.info("α = %.5f  (%d/%d)", alpha, idx + 1, N_ALPHA)

        def get(a):
            if a is None:
                return None
            if a not in cache:
                cache[a] = robust_opt(a, seed=seed_outer)
            return cache[a]

        res_c   = get(alpha)
        H_star  = res_c["H_star"]
        w_star  = res_c["w_star"]
        pi_star = res_c["pi_star"]
        lam_star= res_c["lam_star"]
        subset  = res_c["subset"]
        R_th    = res_c["R"]
        Sigma_th= res_c["Sigma"]
        iters   = res_c["iters"]

        _, _, lambda_log = build_params(alpha)
        lambda_min_val   = float(min(lambda_log))

        prev_alpha = ALPHA_GRID[idx - 1] if idx > 0 else None
        next_alpha = ALPHA_GRID[idx + 1] if idx < N_ALPHA - 1 else None
        h_left  = alpha - prev_alpha if prev_alpha is not None else None
        h_right = next_alpha - alpha if next_alpha is not None else None

        if prev_alpha is not None and next_alpha is not None:
            h = min(h_left, h_right)
            pi_p = get(alpha + h)["pi_star"]
            pi_m = get(alpha - h)["pi_star"]
        elif prev_alpha is None:
            h = h_right
            pi_p = get(next_alpha)["pi_star"]
            pi_m = pi_star
        else:
            h = h_left
            pi_p = pi_star
            pi_m = get(prev_alpha)["pi_star"]

        dpi_star = finite_diff_vec(pi_p, pi_m, h, K)

        cstr_ret = R_th.T @ pi_star if pi_star is not None else np.full(M, np.nan)
        lam_vec  = np.zeros(M)
        if lam_star is not None and subset is not None:
            for j, m_idx in enumerate(subset):
                lam_vec[m_idx] = lam_star[j]

        # --- robust Sharpe と上限 ------------------------------------
        SR_rob, SR_bound = robust_sharpe(pi_star, w_star, R_th, Sigma_th)

        row = ([alpha, H_star,
                int((w_star > 1e-6).sum()) if w_star is not None else np.nan,
                lambda_min_val, iters,
                SR_rob, SR_bound]                    # ← 追加
               + list(w_star) + list(pi_star)
               + list(cstr_ret) + list(lam_vec) + list(dpi_star))
        rows.append(row)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(csv_path, index=False, float_format="%.10g")
    log.info("CSV written to %s  (elapsed %.1fs)", csv_path,
             (datetime.now() - t0).total_seconds())
    return df

# ---------------------------------------------------------------------
# 9. MAIN
# ---------------------------------------------------------------------
if __name__ == "__main__":
    run_alpha_sweep()


========== α-Sweep START ==========
========== α-Sweep START ==========
========== α-Sweep START ==========
========== α-Sweep START ==========
========== α-Sweep START ==========
========== α-Sweep START ==========
========== α-Sweep START ==========


INFO:sweep:========== α-Sweep START ==========


α = 0.01000  (1/501)
α = 0.01000  (1/501)
α = 0.01000  (1/501)
α = 0.01000  (1/501)
α = 0.01000  (1/501)
α = 0.01000  (1/501)
α = 0.01000  (1/501)


INFO:sweep:α = 0.01000  (1/501)


α = 0.01098  (2/501)
α = 0.01098  (2/501)
α = 0.01098  (2/501)
α = 0.01098  (2/501)
α = 0.01098  (2/501)
α = 0.01098  (2/501)
α = 0.01098  (2/501)


...

INFO:sweep:α = 0.41474  (414/501)


α = 0.41572  (415/501)
α = 0.41572  (415/501)
α = 0.41572  (415/501)
α = 0.41572  (415/501)
α = 0.41572  (415/501)
α = 0.41572  (415/501)
α = 0.41572  (415/501)


INFO:sweep:α = 0.41572  (415/501)


α = 0.41670  (416/501)
α = 0.41670  (416/501)
α = 0.41670  (416/501)
α = 0.41670  (416/501)
α = 0.41670  (416/501)
α = 0.41670  (416/501)
α = 0.41670  (416/501)


INFO:sweep:α = 0.41670  (416/501)


#$r^m$のみが異なる場合

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
halfpair_alpha_sweep_SR.py   –   Robust MVP (½–½) α-sweep
2025-05-01  unified-CSV version   (K = M = 4)

* すべてのペア w_i = w_j = ½ を列挙し
      H_{ij}(α) = min_{π∈A} πᵀ V̄ π          (閉形式)
  の **最大** を H_star として返す。
* (π*, w*) から
      SR_rob, SR_bound, cstr_ret_m, dpi*, λ
  を計算し α-grid を CSV 出力する。
* 列順・列名は 2025-04-30 版 α-sweep と同一。

実行:  python halfpair_alpha_sweep_SR.py
"""

# ---------------------------------------------------------------------
# 0. IMPORTS & CONSTANTS
# ---------------------------------------------------------------------
import numpy as np
import itertools, math, logging, sys
import scipy.linalg as la
import pandas as pd
from datetime import datetime

K = M = 4

mu_tilde  = 0.03

ALPHA_GRID   = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)
# 固定パラメータ（σ, C は α 非依存）
r_base = np.array([0.02, 0.03, 0.04, 0.05])
ΔA   =  np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -2, +2, -2],
                    [+2, -2, +2, -2]], dtype=float)
Δr_m = ΔA / 1000
sigma_base = np.array([0.20, 0.25, 0.30, 0.35])

def block_corr(rho_in: float, rho_out: float) -> np.ndarray:
    C = np.full((K, K), rho_out); np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j: C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j: C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
Vbar   = np.diag(sigma_base) @ C_base @ np.diag(sigma_base)         # SPD
P      = la.inv(Vbar)                                               # shared inverse
lambda_min_const = float(np.linalg.eigvalsh(Vbar).min())            # α 非依存

# ---------------------------------------------------------------------
# 1. ½–½ 内側値 H_{ij}(α) と π の閉形式
# ---------------------------------------------------------------------
def inner_halfpair(alpha: float, i: int, j: int):
    """Return (value, π, λ-vector, active_subset)."""
    R = r_base[:, None] + alpha * Δr_m
    r_i, r_j = R[:, i], R[:, j]

    # case (both active)
    A     = np.stack([r_i, r_j], axis=1)            # K×2
    Mmat  = A.T @ P @ A                             # 2×2
    lam2  = mu_tilde * la.solve(Mmat, np.ones(2))
    pi2   = P @ A @ lam2
    val2  = pi2 @ Vbar @ pi2

    # case (only i active)
    pr_i  = P @ r_i;  lam_i = mu_tilde / (r_i @ pr_i)
    pi_i  = lam_i * pr_i
    val_i = val_i_ok = math.inf
    if pi_i @ r_j >= mu_tilde - 1e-12:
        val_i_ok = pi_i @ Vbar @ pi_i

    # case (only j active)
    pr_j  = P @ r_j;  lam_j = mu_tilde / (r_j @ pr_j)
    pi_j  = lam_j * pr_j
    val_j = val_j_ok = math.inf
    if pi_j @ r_i >= mu_tilde - 1e-12:
        val_j_ok = pi_j @ Vbar @ pi_j

    # choose best feasible
    if val2 <= val_i_ok and val2 <= val_j_ok:
        lam_vec = np.zeros(M); lam_vec[[i, j]] = lam2
        return val2, pi2, lam_vec, (i, j)
    if val_i_ok <= val_j_ok:
        lam_vec = np.zeros(M); lam_vec[i] = lam_i
        return val_i_ok, pi_i, lam_vec, (i,)
    lam_vec = np.zeros(M); lam_vec[j] = lam_j
    return val_j_ok, pi_j, lam_vec, (j,)

# ---------------------------------------------------------------------
# 2. SR_rob と SR_bound
# ---------------------------------------------------------------------
def robust_sharpe(pi, w_vec, R):
    r_bar = R @ w_vec
    Sigma_b = Vbar + sum(w_vec[m] * np.outer(R[:, m], R[:, m]) for m in range(M))
    Vw = Sigma_b - np.outer(r_bar, r_bar)
    num = float(pi @ r_bar)
    denom = float(np.sqrt(pi @ Vw @ pi))
    SR_rob = num / denom if denom > 0 else np.nan
    rho_max = max(R[:, m] @ la.solve(Vbar + np.outer(R[:, m], R[:, m]), R[:, m])
                  for m in range(M))
    SR_bound = math.sqrt(rho_max / (1 - rho_max)) if rho_max < 1 else np.nan
    return SR_rob, SR_bound

# ---------------------------------------------------------------------
# 3. α-SWEEP MAIN
# ---------------------------------------------------------------------
def run_alpha_sweep(csv_path="halfpair_opt_results_SR.csv"):
    log = logging.getLogger("sweep")
    log.setLevel(logging.INFO); log.addHandler(logging.StreamHandler(sys.stdout))
    log.info("========== ½–½ α-Sweep START ==========")

    col_w   = [f"w*_m{m}"   for m in range(M)]
    col_pi  = [f"pi*_k{k}"  for k in range(K)]
    col_ret = [f"cstr_ret_m{m}" for m in range(M)]
    col_lam = [f"lam_m{m}"  for m in range(M)]
    col_dpi = [f"dpi*_k{k}" for k in range(K)]
    columns = (["alpha","H_star","supp_w","lambda_min","iterations",
                "SR_rob","SR_bound"]
               + col_w + col_pi + col_ret + col_lam + col_dpi)

    rows, cache = [], {}      # cache[alpha] = dict(pi_star, ...)
    t0 = datetime.now()

    # --- sweep ----------------------------------------------------
    for idx, alpha in enumerate(ALPHA_GRID):
        log.info("α = %.2f  (%d/%d)", alpha, idx + 1, len(ALPHA_GRID))
        # evaluate all pairs, keep max (== H_star)
        best_val = -math.inf; best = None
        for i, j in itertools.combinations(range(M), 2):
            val, pi, lam_vec, subset = inner_halfpair(alpha, i, j)
            if val > best_val + 1e-12:
                best_val = val; best = (i, j, pi, lam_vec)
        i, j, pi_star, lam_vec = best
        w_star = np.zeros(M); w_star[[i, j]] = 0.5
        R = r_base[:, None] + alpha * Δr_m
        SR_rob, SR_bound = robust_sharpe(pi_star, w_star, R)
        cstr_ret = R.T @ pi_star

        cache[alpha] = {"pi_star": pi_star}

        # dpi*: central / forward / backward diff
        if idx == 0:          # forward
            h = ALPHA_GRID[1] - ALPHA_GRID[0]
            pi_p = None
            pi_m = pi_star
            pi_p = (inner_halfpair(ALPHA_GRID[1], *best[:2]))[1]
        elif idx == len(ALPHA_GRID)-1:   # backward
            h = ALPHA_GRID[-1] - ALPHA_GRID[-2]
            pi_p = pi_star
            pi_m = cache[ALPHA_GRID[-2]]["pi_star"]
        else:                 # central
            h_left  = alpha - ALPHA_GRID[idx-1]
            h_right = ALPHA_GRID[idx+1] - alpha
            h = min(h_left, h_right)
            # 確実にキャッシュを埋める
            for a in (ALPHA_GRID[idx-1], ALPHA_GRID[idx+1]):
                if a not in cache:
                    i_,j_,pi_,lam_ = None,None,None,None
                    for p in itertools.combinations(range(M),2):
                        v,p_,l_,sub_=inner_halfpair(a,*p)
                        if i_ is None or v>val_best:
                            val_best=v;i_,j_=p;pi_,l_=p_,l_
                    cache[a]={"pi_star":pi_}
            pi_m = cache[ALPHA_GRID[idx-1]]["pi_star"]
            pi_p = cache[ALPHA_GRID[idx+1]]["pi_star"]
        dpi_star = (pi_p - pi_m) / (2*h) if pi_p is not None else np.full(K, np.nan)

        row = ([alpha, best_val, 2, lambda_min_const, np.nan,
                SR_rob, SR_bound]
               + list(w_star) + list(pi_star)
               + list(cstr_ret) + list(lam_vec) + list(dpi_star))
        rows.append(row)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(csv_path, index=False, float_format="%.10g")
    log.info("CSV written to %s  (elapsed %.1fs)",
             csv_path, (datetime.now() - t0).total_seconds())
    return df

# ---------------------------------------------------------------------
# 4. MAIN
# ---------------------------------------------------------------------
if __name__ == "__main__":
    run_alpha_sweep()


#$\sigma^m$のみが異なる場合

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as la
from datetime import datetime

# ---------------------------- GLOBALS ----------------------------
K, M = 4, 4                              # assets / models
mu_tilde = 0.03                          # target return μ̃

ALPHA_GRID   = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)
# ---------- optimiser hyper-parameters ---------------------------
MAX_OPT_ITERS = 1000
tol_grad = 1e-8
step_size_norm_threshold = 1e-10

# -------------- base return (common across models) --------------
r_bar = np.array([0.02, 0.03, 0.04, 0.05])     # identical

# -------------- volatility base & deviations ----------------------
sigma_base = np.array([0.20, 0.25, 0.30, 0.35])
Delta_A =np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -2, +2, -2],
                    [+2, -2, +2, -2]], dtype=float)
Delta_sigma_m = Delta_A / 100                # Δσ per model

# ---------------- correlation: two-sector block -------------------
def block_corr(rho_in, rho_out):
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    for i, j in ((0, 1), (2, 3)):
        C[i, j] = C[j, i] = rho_in
    return C

C_base = block_corr(0.75, 0.50)
tilde_C_list = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]
beta_corr = 0.5

# ---------- correlation blending + SPD projection -----------------
def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4):
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval = np.clip(eigval, eps, None)
    Cs = eigvec @ np.diag(eigval) @ eigvec.T
    d = np.sqrt(np.diag(Cs))
    return Cs / np.outer(d, d)        # correlation matrix

# ---------------------- simplex projection ------------------------
def proj_simplex(v):
    """Projection onto Δ_M."""
    v = np.asarray(v, float)
    if (v >= 0).all() and np.isclose(v.sum(), 1.0, atol=1e-10):
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - 1
    rho = np.where(u - cssv / (np.arange(len(u)) + 1) > 0)[0][-1]
    theta = cssv[rho] / (rho + 1)
    return np.maximum(v - theta, 0.0)

# ---------------------- parameter builder -------------------------
def build_params(alpha):
    Σ = np.zeros((K, K, M))
    C_list = [mix_corr(C_base, tilde_C_list[m], alpha, beta_corr) for m in range(M)]

    for m in range(M):
        σ = sigma_base + alpha * Delta_sigma_m[:, m]
        # Use common correlation matrix C_base for all models, but vary sigma
        # This follows Proposition 2 where r^m and C^m are common across models
        V = np.diag(σ) @ C_base @ np.diag(σ)
        Σ[:, :, m] = V + np.outer(r_bar, r_bar)    # Σ^m
    return Σ

# ---------------- outer objective & gradient ----------------------
def phi_and_grad(w, Σ):
    Vw = sum(w[m] * Σ[:, :, m] for m in range(M)) - np.outer(r_bar, r_bar)

    # Add a small regularization to ensure positive definiteness
    reg_factor = 1e-8
    Vw_reg = Vw + reg_factor * np.eye(K)

    # Solve the linear system
    try:
        y = la.solve(Vw_reg, r_bar, assume_a='pos')
        phi = r_bar @ y
        grad = np.array([-y @ Σ[:, :, m] @ y for m in range(M)])
    except np.linalg.LinAlgError:
        # If still singular, use pseudoinverse
        y = np.linalg.pinv(Vw_reg) @ r_bar
        phi = r_bar @ y
        grad = np.array([-y @ Σ[:, :, m] @ y for m in range(M)])

    return phi, grad

# -------------- projected-gradient (BB step) ----------------------
def minimise_phi(Σ, w0=None):
    w = proj_simplex(w0 if w0 is not None else np.ones(M) / M)
    phi, g = phi_and_grad(w, Σ)
    lr = 0.2
    for it in range(1, MAX_OPT_ITERS + 1):
        if np.linalg.norm(g) < tol_grad:
            break
        # Barzilai–Borwein step size (first-order)
        if it > 1:
            s, y = w - w_prev, g - g_prev
            lr = np.clip((s @ y) / (y @ y + 1e-12), 1e-4, 0.5)
        w_prev, g_prev = w.copy(), g.copy()
        # line search (simple back-tracking)
        for _ in range(20):
            w_new = proj_simplex(w - lr * g)
            phi_new, g_new = phi_and_grad(w_new, Σ)
            if phi_new < phi - 1e-10:
                w, phi, g = w_new, phi_new, g_new
                break
            lr *= 0.5
        else:
            break
    return phi, w, it

# --------------------------- α-sweep ------------------------------
def run_sweep(out_csv="sigma_only_opt_results.csv"):
    cols_w   = [f"w*_m{m}" for m in range(M)]
    cols_pi  = [f"pi*_k{k}" for k in range(K)]
    cols_ret = [f"cstr_ret_m{m}" for m in range(M)]
    cols_lam = [f"lam_m{m}" for m in range(M)]
    cols_dpi = [f"dpi*_k{k}" for k in range(K)]

    records = []
    prev_pi_star = None

    for idx, alpha in enumerate(ALPHA_GRID):
        Σ = build_params(alpha)
        phi_star, w_star, nit = minimise_phi(Σ)

        # Calculate V^w (model-weighted covariance)
        Vw = sum(w_star[m] * Σ[:, :, m] for m in range(M)) - np.outer(r_bar, r_bar)

        # Add small regularization to ensure stability
        reg_factor = 1e-8
        Vw_reg = Vw + reg_factor * np.eye(K)

        # Calculate minimum eigenvalue for diagnostics
        try:
            λ_min = np.min(np.linalg.eigvalsh(Vw))
        except np.linalg.LinAlgError:
            λ_min = np.nan

        # Calculate optimal portfolio (π*) according to Proposition 2
        try:
            Vw_inv = np.linalg.inv(Vw_reg)
            r_Vinv_r = r_bar @ Vw_inv @ r_bar
            pi_star = (mu_tilde * Vw_inv @ r_bar) / r_Vinv_r
        except np.linalg.LinAlgError:
            # Fallback to pseudoinverse if inversion fails
            Vw_inv = np.linalg.pinv(Vw_reg)
            r_Vinv_r = r_bar @ Vw_inv @ r_bar
            pi_star = (mu_tilde * Vw_inv @ r_bar) / r_Vinv_r

        # Calculate objective value H*
        H_star = mu_tilde**2 / phi_star

        # Calculate dpi_star using finite difference
        if idx > 0 and prev_pi_star is not None:
            alpha_diff = alpha - ALPHA_GRID[idx-1]
            dpi_star = (pi_star - prev_pi_star) / alpha_diff
        else:
            dpi_star = np.zeros(K)  # For the first iteration

        # Calculate Robust Sharpe Ratio (correctly using pi_star)
        try:
            denom = np.sqrt(pi_star @ Vw @ pi_star)
            SR_rob = (pi_star @ r_bar) / denom if denom > 0 else np.nan
        except:
            SR_rob = np.nan

        # Calculate SR bound
        try:
            rho_max = max(
                r_bar @ np.linalg.pinv(Σ[:, :, m]) @ r_bar
                for m in range(M)
            )
            SR_bound = np.sqrt(rho_max / (1 - rho_max)) if rho_max < 1 else np.nan
        except:
            SR_bound = np.nan

        # Create record for this alpha value
        rec = dict(
            alpha=alpha,
            H_star=H_star,
            supp_w=int((w_star > 1e-6).sum()),
            lambda_min=λ_min,
            iterations=nit,
            SR_rob=SR_rob,
            SR_bound=SR_bound
        )

        # Add weight and portfolio allocation values
        rec.update({f"w*_m{m}": w_star[m] for m in range(M)})
        rec.update({f"pi*_k{k}": pi_star[k] for k in range(K)})

        # Add derivative values (correctly as a vector)
        rec.update({f"dpi*_k{k}": dpi_star[k] for k in range(K)})

        # Add placeholders for other columns
        rec.update({k: np.nan for k in cols_ret + cols_lam})

        records.append(rec)
        prev_pi_star = pi_star.copy()  # Save for next iteration

    cols = (["alpha","H_star","supp_w","lambda_min","iterations",
             "SR_rob","SR_bound"]+cols_w+cols_pi+cols_ret+cols_lam+cols_dpi)
    df = pd.DataFrame.from_records(records, columns=cols)
    df.to_csv(out_csv, index=False, float_format="%.10g")
    print(f"[{datetime.now().isoformat(timespec='seconds')}]  CSV written → {out_csv}")
    return df


if __name__ == "__main__":
    df_preview = run_sweep()
    print(df_preview.head())      # Colab/Jupyter でテーブル表示

#$C^m$のみが異なる場合

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

# -------------------- problem constants --------------------------
K = M = 4
mu_tilde = 0.03

ALPHA_GRID = np.linspace(ALPHA_MIN, ALPHA_MAX, N_ALPHA)

r_bar = np.array([0.02, 0.03, 0.04, 0.05])
sigma_bar = np.array([0.20, 0.25, 0.30, 0.35])
inv_sigma_bar = 1.0 / sigma_bar          # D(σ̄)⁻¹
g_vec = r_bar / sigma_bar                # g = D(σ̄)⁻¹ r̄

beta_corr = 0.5                          # mixing weight

# ---- base / tilde correlation matrices --------------------------
def block_corr(rho_in, rho_out):
    C = np.full((K, K), rho_out)
    np.fill_diagonal(C, 1.0)
    sectA, sectB = [0, 1], [2, 3]
    for i in sectA:
        for j in sectA:
            if i != j:
                C[i, j] = rho_in
    for i in sectB:
        for j in sectB:
            if i != j:
                C[i, j] = rho_in
    return C

C_base = block_corr(0.75, 0.50)          # common base
C_tilde = [
    block_corr(0.85, 0.40),
    block_corr(0.65, 0.40),
    block_corr(0.65, 0.60),
    block_corr(0.85, 0.60),
]

# ---- blend & SPD-project ----------------------------------------
def mix_corr(C0, C1, alpha, beta=0.5, eps=1e-4):
    C = (1 - beta * alpha) * C0 + beta * alpha * C1
    eigval, eigvec = np.linalg.eigh(C)
    eigval = np.clip(eigval, eps, None)        # SPD clip
    C_spd = eigvec @ np.diag(eigval) @ eigvec.T
    D = np.sqrt(np.diag(C_spd))
    return C_spd / np.outer(D, D)              # correlation

# -------- simplex projection (Condat/Fukushima) ------------------
def proj_simplex(v):
    v = v.clip(min=0.0)
    if abs(v.sum() - 1) < 1e-10:
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - 1
    rho = (u - cssv / (np.arange(len(v)) + 1) > 0).nonzero()[0][-1]
    theta = cssv[rho] / (rho + 1)
    return np.maximum(v - theta, 0.0)

# -------- Φ(w) = gᵀQ⁻¹g  and its gradient ∇Φ(w) ------------------
def phi_and_grad(w, Cstack):
    Q = sum(w[m] * Cstack[m] for m in range(M))
    y = np.linalg.solve(Q, g_vec)             # Cholesky solve (4×4)
    phi = g_vec @ y
    grad = -np.array([y @ Cstack[m] @ y for m in range(M)])
    return phi, grad, y

# -------------- fast L-BFGS on the simplex Δ_M -------------------
def minimise_phi(Cstack, it_max=40, lbfgs_m=5):
    w = np.full(M, 1 / M)
    s_hist, y_hist = [], []
    phi, grad, y = phi_and_grad(w, Cstack)

    for _ in range(it_max):
        # ---- two-loop recursion (limited memory) ---------------
        q = grad.copy()
        alpha_hist = []
        for s, y_h in reversed(list(zip(s_hist, y_hist))):
            rho = 1.0 / (y_h @ s)
            a = rho * (s @ q)
            alpha_hist.append(a)
            q -= a * y_h
        if s_hist:
            gamma = (s_hist[-1] @ y_hist[-1]) / (y_hist[-1] @ y_hist[-1])
            q *= gamma
        for s, y_h, a in zip(s_hist, y_hist, reversed(alpha_hist)):
            rho = 1.0 / (y_h @ s)
            beta = rho * (y_h @ q)
            q += s * (a - beta)
        d = -q

        # ---- Armijo line search on simplex ---------------------
        step = 1.0
        while step > 1e-6:
            w_new = proj_simplex(w + step * d)
            phi_new, grad_new, y_new = phi_and_grad(w_new, Cstack)
            if phi_new < phi - 1e-4 * step * (grad @ d):
                break
            step *= 0.5
        if step <= 1e-6:
            break

        s_k = w_new - w
        y_k = grad_new - grad
        if y_k @ s_k > 1e-12:
            s_hist.append(s_k)
            y_hist.append(y_k)
            if len(s_hist) > lbfgs_m:
                s_hist.pop(0); y_hist.pop(0)

        w, phi, grad, y = w_new, phi_new, grad_new, y_new
        if np.linalg.norm(proj_simplex(w - grad) - w) < 1e-8:
            break
    return phi, w, y

# ---------------------------------------------------------------------
# X.  ROBUST SHARPE と 上限を計算
# ---------------------------------------------------------------------
def robust_sharpe(pi, w, R, Sigma):
    """SR_rob と理論上限 SR_bound を返す。"""
    if pi is None or w is None:
        return np.nan, np.nan
    r_bar = R @ w
    Sigma_b = sum(w[m] * Sigma[:, :, m] for m in range(M))
    Vw = Sigma_b - np.outer(r_bar, r_bar)
    num = pi @ r_bar
    denom = np.sqrt(pi @ Vw @ pi)
    SR_rob = num / denom if denom > 0 else np.nan

    # ρ_max
    rho_max = max(
        r_bar @ np.linalg.pinv(Sigma[:, :, m]) @ r_bar
        for m in range(M)
    )
    SR_bound = (np.sqrt(rho_max / (1 - rho_max))
                if rho_max < 1 else np.nan)

    return SR_rob, SR_bound

# ------------------------ main computation -----------------------
rows = []
prev_pi_star = None  # For derivative calculation

for idx, alpha in enumerate(ALPHA_GRID):
    # Cstack depends on α
    Cstack = [mix_corr(C_base, C_tilde[m], alpha, beta_corr) for m in range(M)]
    phi_star, w_star, y_star = minimise_phi(Cstack)
    H_star = mu_tilde ** 2 / phi_star
    # π* = μ̃ / Φ* · D(σ̄)⁻¹ y
    pi_star = (mu_tilde / phi_star) * inv_sigma_bar * y_star
    cstr_ret = r_bar.T @ pi_star   # Constraint returns

    # --- Compute covariance matrices correctly ---
    V = np.zeros((K, K, M))
    Sigma = np.zeros((K, K, M))

    for m in range(M):
        C_m = mix_corr(C_base, C_tilde[m], alpha, beta_corr)
        # V^m = D(σ̄)C^mD(σ̄)
        V[:, :, m] = np.diag(sigma_bar) @ C_m @ np.diag(sigma_bar)
        # Σ^m = V^m + r^m(r^m)^T
        Sigma[:, :, m] = V[:, :, m] + np.outer(r_bar, r_bar)

    # --- Calculate V^w and its minimum eigenvalue ---
    Vw = sum(w_star[m] * V[:, :, m] for m in range(M))
    # Find minimum eigenvalue of V^w
    lambda_min = np.min(np.linalg.eigvalsh(Vw))

    # --- Calculate derivative of pi* with respect to alpha using finite difference ---
    dpi_star = np.zeros(K)
    if idx > 0 and prev_pi_star is not None:
        alpha_diff = alpha - ALPHA_GRID[idx-1]
        dpi_star = (pi_star - prev_pi_star) / alpha_diff

    # --- robust Sharpe and bound --------------------------------
    # 修正: r_barは全てのモデルで共通なので、単一ベクトルで計算
    SR_rob, SR_bound = robust_sharpe(pi_star, w_star, np.tile(r_bar.reshape(K, 1), M), Sigma)

    # Ensure all variables are in correct iterable form
    row = [alpha, H_star, lambda_min, SR_rob, SR_bound] + list(w_star) + list(pi_star) + [cstr_ret] + list(dpi_star)
    rows.append(row)

    # Store current pi_star for next iteration's derivative calculation
    prev_pi_star = pi_star.copy()

# --------------------------- CSV export --------------------------
columns = ["alpha", "H_star", "lambda_min", "SR_rob", "SR_bound",
           "w*_m0", "w*_m1", "w*_m2", "w*_m3",
           "pi*_k0", "pi*_k1", "pi*_k2", "pi*_k3",
           "cstr_ret",
           "dpi*_k0", "dpi*_k1", "dpi*_k2", "dpi*_k3"]

df = pd.DataFrame(rows, columns=columns)
out_path = "phi_min_results_with_SR.csv"
df.to_csv(out_path, index=False)

print("CSV saved →", out_path)
print("Finished:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))